In [1]:
%cd C:/Users/alapa/funiegan/

C:\Users\alapa\funiegan


C:\Users\alapa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import cv2
import numpy as np
from skimage import exposure

def calculate_image_metrics(image):
    """Calculate basic metrics to assess image quality."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast = gray.std()  # Standard deviation as a measure of contrast
    noise_level = np.mean(cv2.Laplacian(gray, cv2.CV_64F)**2)  # Variance of Laplacian
    return contrast, noise_level

def apply_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    """Apply CLAHE for contrast enhancement."""
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    enhanced = cv2.merge((l, a, b))
    return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

def denoise_image(image, method="bilateral", d=9, sigma_color=75, sigma_space=75):
    """Denoise the image using the specified method."""
    if method == "bilateral":
        return cv2.bilateralFilter(image, d, sigma_color, sigma_space)
    elif method == "median":
        return cv2.medianBlur(image, 5)
    else:
        raise ValueError("Unsupported denoising method.")

def dehaze_image(image, beta=1.0):
    """Simple dehazing using dark channel prior."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dark_channel = cv2.min(cv2.min(image[:, :, 0], image[:, :, 1]), image[:, :, 2])
    transmission = 1 - beta * (dark_channel / 255)
    transmission = np.clip(transmission, 0.1, 1)
    result = image / transmission[:, :, None]
    return np.uint8(np.clip(result, 0, 255))

def restore_red_channel(image, red_boost=1.5):
    """Restore the red channel for underwater images."""
    b, g, r = cv2.split(image)
    r = cv2.addWeighted(r, red_boost, 0, 0, 0)
    return cv2.merge((b, g, r))

def process_underwater_image(image_path, save_path=None):
    """Main pipeline for underwater image processing."""
    # Read input image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Step 1: Assess image quality
    contrast, noise_level = calculate_image_metrics(image)
    print(f"Initial Contrast: {contrast:.2f}, Noise Level: {noise_level:.2f}")

    # Step 2: Preprocessing - Denoising
    if noise_level > 1000:  # Threshold for high noise
        print("Applying denoising...")
        image = denoise_image(image, method="bilateral")

    # Step 3: Enhancement - Dehazing
    print("Applying dehazing...")
    image = dehaze_image(image)

    # Step 4: Enhancement - Contrast Adjustment
    if contrast < 50:  # Threshold for low contrast
        print("Applying CLAHE for contrast enhancement...")
        image = apply_clahe(image)

    # Step 5: Postprocessing - Red Channel Restoration
    print("Restoring red channel...")
    image = restore_red_channel(image)

    # Save and return the processed image
    if save_path:
        cv2.imwrite(save_path, image)
        print(f"Processed image saved to {save_path}")

    return original_image, image

# Example usage
#input_image = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainA/264286_00007889.jpg"  # Replace with your image path
input_image = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x8.pth/output_264286_00007889.jpg"
output_image = "C:/Users/alapa/funiegan/nonAIprocessed/processed_underwater1.jpg"

original, processed = process_underwater_image(input_image, save_path=output_image)

# Display results
cv2.imshow("Original Image", original)
cv2.imshow("Processed Image", processed)
cv2.waitKey(0)
cv2.destroyAllWindows()


Initial Contrast: 68.71, Noise Level: 289.04
Applying dehazing...
Restoring red channel...
Processed image saved to C:/Users/alapa/funiegan/nonAIprocessed/processed_underwater1.jpg


In [ ]:
import cv2
import numpy as np

def calculate_image_metrics(image):
    """Calculate basic metrics to assess image quality."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contrast = gray.std()  # Standard deviation as a measure of contrast
    noise_level = np.mean(cv2.Laplacian(gray, cv2.CV_64F)**2)  # Variance of Laplacian
    return contrast, noise_level

def apply_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    """Apply CLAHE for contrast enhancement."""
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    l = clahe.apply(l)
    enhanced = cv2.merge((l, a, b))
    return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

def denoise_image(image, method="bilateral", d=9, sigma_color=75, sigma_space=75):
    """Denoise the image using the specified method."""
    if method == "bilateral":
        return cv2.bilateralFilter(image, d, sigma_color, sigma_space)
    elif method == "median":
        return cv2.medianBlur(image, 5)
    else:
        raise ValueError("Unsupported denoising method.")

def dehaze_image(image, beta=0.9):
    """Simplified dehazing based on dark channel prior."""
    dark_channel = np.min(image, axis=2)
    transmission = 1 - beta * (dark_channel / 255)
    transmission = np.clip(transmission, 0.3, 1)  # Avoid division by zero
    result = image / transmission[:, :, None]
    return np.uint8(np.clip(result, 0, 255))

def restore_red_channel(image, red_boost=1.2):
    """Restore the red channel for underwater images."""
    b, g, r = cv2.split(image)
    r = np.clip(r * red_boost, 0, 255).astype(np.uint8)
    return cv2.merge((b, g, r))

def process_underwater_image(image_path, save_path=None, debug=False):
    """Main pipeline for underwater image processing."""
    # Read input image
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Step 1: Assess image quality
    contrast, noise_level = calculate_image_metrics(image)
    if debug:
        print(f"Initial Contrast: {contrast:.2f}, Noise Level: {noise_level:.2f}")

    # Step 2: Preprocessing - Denoising
    if noise_level > 500:  # Adjusted threshold for high noise
        if debug: print("Applying denoising...")
        image = denoise_image(image, method="bilateral")

    # Step 3: Enhancement - Dehazing
    if debug: print("Applying dehazing...")
    image = dehaze_image(image)

    # Step 4: Enhancement - Contrast Adjustment
    if contrast < 40:  # Adjusted threshold for low contrast
        if debug: print("Applying CLAHE for contrast enhancement...")
        image = apply_clahe(image)

    # Step 5: Postprocessing - Red Channel Restoration
    if debug: print("Restoring red channel...")
    image = restore_red_channel(image)

    # Save and return the processed image
    if save_path:
        cv2.imwrite(save_path, image)
        if debug: print(f"Processed image saved to {save_path}")

    return original_image, image

# Example usage
#input_image = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainA/264286_00007889.jpg"  # Replace with your image path
input_image = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x8.pth/output_264286_00007889.jpg"
output_image = "C:/Users/alapa/funiegan/nonAIprocessed/processed_underwater1.jpg"
# Enable debug mode for detailed output
debug_mode = True

# Process the image
original, processed = process_underwater_image(input_image, save_path=output_image, debug=debug_mode)

# Display results
cv2.imshow("Original Image", original)
cv2.imshow("Processed Image", processed)
cv2.waitKey(0)  # Automatically close after 5 seconds
cv2.destroyAllWindows()


Initial Contrast: 68.71, Noise Level: 289.04
Applying dehazing...
Restoring red channel...
Processed image saved to C:/Users/alapa/funiegan/nonAIprocessed/processed_underwater1.jpg


In [ ]:
import cv2
import numpy as np

def enhance_contrast(image):
    """Apply Multi-Scale Retinex (MSR) for illumination correction."""
    img_float = np.float32(image) + 1.0  # Convert to float for calculation
    # Apply Gaussian blur and log operation for Retinex processing
    retinex = np.log10(img_float) - np.log10(cv2.GaussianBlur(img_float, (81, 81), 30))
    
    # Clip to avoid excessive brightening
    retinex = np.clip((retinex - np.min(retinex)) / (np.max(retinex) - np.min(retinex)) * 255, 0, 255)
    return np.uint8(retinex)

def color_correct(image, red_boost=1.1, blue_boost=1.1):
    """Correct color channels by boosting red and blue."""
    b, g, r = cv2.split(image)
    
    # Boost channels but clip values to avoid overexposure
    r = np.clip(r * red_boost, 0, 255).astype(np.uint8)
    b = np.clip(b * blue_boost, 0, 255).astype(np.uint8)
    
    return cv2.merge((b, g, r))

def denoise(image):
    """Apply noise reduction using Non-Local Means Denoising."""
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)

def dehaze(image, beta=0.85):
    """Dehaze using Dark Channel Prior and a controlled transmission map."""
    # Dark channel computation (min across channels)
    dark_channel = np.min(image, axis=2)
    transmission = 1 - beta * (dark_channel / 255)
    
    # Clip the transmission to avoid over-enhancement
    transmission = np.clip(transmission, 0.1, 1)  # Limit transmission to avoid extreme results
    
    # Apply dehazing using a physical model (adjust transmission to control effect)
    dehazed = (image - 0.1) / transmission[:, :, None]  # Prevent extreme increase
    dehazed = np.clip(dehazed, 0, 255)  # Clip to valid range
    
    return np.uint8(dehazed)

def super_resolution(image, scale=2, model_path="nonAIprocessed/models/EDSR_x2.pb"):
    """Apply Super-Resolution using OpenCV's DNN SuperRes."""
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    sr.readModel(model_path)  # Load the pre-trained model
    sr.setModel("edsr", scale)  # Use the EDSR model with scaling factor
    result = sr.upsample(image)

    # Clip the result to ensure pixel values are within valid range
    result = np.clip(result, 0, 255)
    return result

def process_underwater_image(image_path, save_path=None):
    """Full underwater image processing pipeline."""
    image = cv2.imread(image_path)
    original = image.copy()

    # Step 1: Enhance contrast
    print("Enhancing contrast...")
    image = enhance_contrast(image)
    cv2.imshow("After Contrast Enhancement", image)
    
    # Step 2: Color correction
    print("Correcting color...")
    image = color_correct(image)
    cv2.imshow("After Color Correction", image)

    # Step 3: Denoising
    print("Denoising...")
    image = denoise(image)
    cv2.imshow("After Denoising", image)

    # Step 4: Dehazing
    print("Dehazing...")
    image = dehaze(image)
    cv2.imshow("After Dehazing", image)

    # Step 5: Super-Resolution
    print("Applying Super-Resolution...")
    image = super_resolution(image)
    cv2.imshow("After Super-Resolution", image)

    # Final safeguard to ensure pixel values stay within the valid range
    image = np.clip(image, 0, 255)
    
    if save_path:
        cv2.imwrite(save_path, image)
        print(f"Processed image saved at {save_path}")

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return original, image


# Example usage
input_image = "C:/Users/alapa/funiegan/EUVP/Paired/underwater_dark/trainA/264286_00007889.jpg"  # Replace with your image path
#input_image = "C:/Users/alapa/imageupscaler/upscalar(ESRGAN)/results/RealESRGAN_x8.pth/output_264286_00007889.jpg"
output_image = "C:/Users/alapa/funiegan/nonAIprocessed/processed_underwater2.jpg"

original, processed = process_underwater_image(input_image, save_path=output_image)

cv2.imshow("Original", original)
cv2.imshow("Processed", processed)
cv2.waitKey(0)
cv2.destroyAllWindows()


Enhancing contrast...
Correcting color...
Denoising...
Dehazing...
Applying Super-Resolution...
Processed image saved at C:/Users/alapa/funiegan/nonAIprocessed/processed_underwater2.jpg
